Before we do any work, we need to import several functions from cdapython:
- `Q` and `query` which power the search
- `columns` which lets us view entity field names
- `unique_terms` which lets view entity field contents

We're also asking cdapython to report it's version so we can be sure we're using the one we mean to.

In [1]:
from cdapython import Q, columns, unique_terms,query, constantVariables
import cdapython
print(cdapython.__file__)
print(cdapython.__version__)
integration_host = "http://35.192.60.10:8080/"
Q.set_host_url(integration_host)

/Users/amanda.charbonneau/github/cda-python/cdapython/__init__.py
2022.5.23


CDA data comes from three sources:
- The [Proteomic Data Commons](https://proteomic.datacommons.cancer.gov/pdc/) (PDC)
- The [Genomic Data Commons](https://gdc.cancer.gov/) (GDC)
- The [Imaging Data Commons](https://datacommons.cancer.gov/repository/imaging-data-commons) (IDC)

The CDA makes this data searchable in four main endpoints:

- `subject`: Specific, unique, individuals
- `researchsubject`: Study-individual aggregate entities. A Subject who was part of three studies will appear as three ResearchSubjects
- `specimen`: Samples taken from individual
- `file`: Data about Subjects, ResearchSubjects, Specimens, and their associated information

and two endpoints that offer deeper information about data in the researchsubject endpoint:

- `diagnosis`: Information about what medical diagnosis a researchsubject has
- `treatment`: Information about what medical treatment(s) were performed for a given diagnosis

If you are looking to build a cohort of distinct individuals who meet some criteria, search by `subject`. If you want to build a cohort, but are particularly interested in studies rather than the participates per se, search by `researchsubject`. If you are looking for biosamples that can be ordered or a specfic format of information (for e.g. histological slides) start with `specimen`. If you are primarily looking for files you can reuse for your own analysis, start with `file`.

In CDA search, these concepts can also be chained together, so you can look specifically for specimen subjects, or researchsubject diagnoses. In the four 'main' tables, all of the rows will have one or more files associated with them that can be directly found by chaining, as in specimen files. Diagnosis and treatment do not have files directly associated with them and so can only be used to find files in conjunction with the other searches.

In all cases, any search can use any metadata field, the only difference between search types is what type of data you return by default. 



## Basic search with endpoints

Let's try a broad search of the CDA to see what information exists about cancers that were first diagnosed in the brain. To run this simple search, we would first construct a query in `Q` and save it to a variable `myquery`:

In [2]:
myquery = Q('ResearchSubject.primary_diagnosis_site = "brain"')


<div style="background-color:#6ce6b9;color:black;padding:20px;">
<h3>Where did those terms come from?</h3>
    
If you aren't sure how we knew what terms to put in our search, please refer back to the <a href="../SearchTerms">What search terms are available?</a> notebook. 
</div>

### subject
Now we can use that query to search any of information types. Let's start by looking at what Subjects meet our criteria. To do that, we will send our query to the subject endpoint, then ask for it to run:

In [3]:
subjectresults = myquery.subject.run()

Getting results from database

Total execution time: 3443 ms


We saved the output in a variable `subjectresults`, so we don't get much visible output. To see what our results are, we need to look into the variable. The simplest way is to call `subjectresults` directly:

In [4]:
subjectresults


            QueryID: 6984f850-0b78-4ac0-973d-068bd6bfe2b1
            Query:SELECT results.* EXCEPT(rn) FROM (SELECT ROW_NUMBER() OVER (PARTITION BY all_Subjects_v3_0_w_RS.id) as rn, all_Subjects_v3_0_w_RS.id AS id, all_Subjects_v3_0_w_RS.identifier AS identifier, all_Subjects_v3_0_w_RS.species AS species, all_Subjects_v3_0_w_RS.sex AS sex, all_Subjects_v3_0_w_RS.race AS race, all_Subjects_v3_0_w_RS.ethnicity AS ethnicity, all_Subjects_v3_0_w_RS.days_to_birth AS days_to_birth, all_Subjects_v3_0_w_RS.subject_associated_project AS subject_associated_project, all_Subjects_v3_0_w_RS.vital_status AS vital_status, all_Subjects_v3_0_w_RS.age_at_death AS age_at_death, all_Subjects_v3_0_w_RS.cause_of_death AS cause_of_death FROM gdc-bq-sample.dev.all_Subjects_v3_0_w_RS AS all_Subjects_v3_0_w_RS LEFT JOIN UNNEST(all_Subjects_v3_0_w_RS.ResearchSubject) AS _ResearchSubject WHERE (UPPER(_ResearchSubject.primary_diagnosis_site) = UPPER('brain'))) as results WHERE rn = 1
            Offset: 0
      

This output tells us our QueryID, which we don't really need, but the computer does to track our questions. Then it tells us four parameters that describe our results:

---

- **Offset:** This is how many rows of information we've told the query to skip in the data, here we didn't tell it to skip anything, so the offset is zero
- **Count:** This is how many rows the current page of our results table has. To keep searches fast, we default to pages with 100 rows.
- **Total Row Count:** This is how many rows are in the full results table
- **More pages:** This is always a True or False. False means that our current page has all the available results. True means that we will see only the first 100 results in this table, and will need to page through for more.

---
    
Now that we've seen the metadata about our results, let's look at the actual table. The easiest way to do this is by using the python function `.to_dataframe()` on our `subjectresults` variable:

In [5]:
subjectresults.to_dataframe()

,id,identifier,species,sex,race,ethnicity,days_to_birth,subject_associated_project,vital_status,age_at_death,cause_of_death
0,900-00-5412,"[{'system': 'IDC', 'value': '900-00-5412'}]",Homo sapiens,None,None,None,None,[rembrandt],None,None,None
1,ACRIN-DSC-MR-Brain-020,"[{'system': 'IDC', 'value': 'ACRIN-DSC-MR-Brai...",Homo sapiens,None,None,None,None,[acrin_dsc_mr_brain],None,None,None
2,ACRIN-DSC-MR-Brain-073,"[{'system': 'IDC', 'value': 'ACRIN-DSC-MR-Brai...",Homo sapiens,None,None,None,None,[acrin_dsc_mr_brain],None,None,None
3,ACRIN-DSC-MR-Brain-116,"[{'system': 'IDC', 'value': 'ACRIN-DSC-MR-Brai...",Homo sapiens,None,None,None,None,[acrin_dsc_mr_brain],None,None,None
4,C136284,"[{'system': 'PDC', 'value': 'C136284'}]",Homo sapiens,male,white,not hispanic or latino,None,[Proteogenomic Analysis of Pediatric Brain Can...,Alive,None,Not Reported
...,...,...,...,...,...,...,...,...,...,...,...
95,TCGA-DB-A64W,"[{'system': 'GDC', 'value': 'TCGA-DB-A64W'}]",Homo sapiens,female,white,not hispanic or latino,-23936,[TCGA-LGG],Dead,438,None
96,TCGA-FG-A4MT,"[{'system': 'GDC', 'value': 'TCGA-FG-A4MT'}, {...",Homo sapiens,female,white,not hispanic or latino,-10082,"[TCGA-LGG, tcga_lgg]",Alive,None,None
97,TCGA-TM-A84I,"[{'system': 'GDC', 'value': 'TCGA-TM-A84I'}]",Homo sapiens,male,white,not hispanic or latino,-11221,[TCGA-LGG],Alive,None,None
98,900-00-5396,"[{'system': 'IDC', 'value': '900-00-5396'}]",Homo sapiens,None,None,None,None,[rembrandt],None,None,None


By default `to_dataframe()` shows us the first and last five rows for the first page of our results, so we can easily preview our data.

Since we queried the Subject endpoint, our default results tell us Subject level information, that is, information about unique individuals: their sex, race, age, species, etc. The `id` column tells us the unique identifier for each individual. The identifier column has nested information about what study or studies a Subject participated in, and will list all of their researchsubject identifiers. 

The `to_dataframe()` function converts the results to a pandas dataframe. So if we save the dataframe to a variable, we can use any pandas functions to explore it. For example, lets see whether any of the Subjects in our first 100 results are black or african american. First we'll save the results to a dataframe, then subset that dataframe to only show rows where the word "black" appears in the "race" column. "NAs" which are shown as "None" in these tables, so for our filter to work, we'll need to specifically tell it to ignore NAs. We're also telling it we want the word "black" regardless of capitalization with `case=False`:


In [6]:
subjectdata = subjectresults.to_dataframe()
subjectdata[subjectdata['race'].str.contains("black", case=False, na=False)]

,id,identifier,species,sex,race,ethnicity,days_to_birth,subject_associated_project,vital_status,age_at_death,cause_of_death
68,TCGA-E1-A7Z3,"[{'system': 'GDC', 'value': 'TCGA-E1-A7Z3'}]",Homo sapiens,female,black or african american,not hispanic or latino,-11330,[TCGA-LGG],Dead,2235,None
90,TCGA-06-A5U0,"[{'system': 'GDC', 'value': 'TCGA-06-A5U0'}]",Homo sapiens,female,black or african american,not hispanic or latino,-7958,[TCGA-GBM],Alive,None,None


There are three subjects in our first hundred results that meet the criteria. If we just want to be sure that the data contains some value, this might be good enough. But often we want to search the entire set of results and not just the first page. We'll cover how to work with large results dataframes in the [Pagination](../Pagination.ipynb) notebook.


---

<div style="background-color:#a2f2ed;color:black;padding:20px;">

<h3>Subject Field Definitions</h3>

<i>A subject is a specific, unique individual: for e.g. a single human. When consent allows, a given entity will have a single subject ID that can be connected to all their studies and data across all datasets</i>

    
<ul>
  <li><b>id:</b> The unique identifier for this subject</li>
  <li><b>identifier:</b> An embedded array of information that includes the originating data center and the ID the subject had there</li>
  <li><b>species:</b> The species of the subject</li>
  <li><b>sex:</b> The sex of the subject </li>
  <li><b>race:</b> The race of the subject</li>
  <li><b>ethnicity:</b> The ethnicity of the subject</li>
  <li><b>days_to_birth:</b> A number counting back to birth from date of first enrollment in a project. Usually negative</li>
  <li><b>subject_associated_project:</b> An embedded array of the names of projects (studies) the subject was part of</li>
  <li><b>vital_status:</b> Whether the subject is alive</li>
  <li><b>age_at_death:</b> The number of days after first enrollment that the subject died</li>
  <li><b>cause_of_death:</b> The cause of death, if known</li>
</ul>  

</div>
    
---

### researchsubject

If we're interested in what researchsubjects meet our critera, we can also run our query against the researchsubject endpoint:

In [7]:
researchsubjectresults = myquery.researchsubject.run()
researchsubjectresults

Getting results from database

Total execution time: 3385 ms



            QueryID: b00cb7ae-8908-422f-88e8-f760e65e5be9
            Query:SELECT results.* EXCEPT(rn) FROM (SELECT ROW_NUMBER() OVER (PARTITION BY _ResearchSubject.id, all_Subjects_v3_0_w_RS.id) as rn, _ResearchSubject.id AS id, _ResearchSubject.identifier AS identifier, _ResearchSubject.member_of_research_project AS member_of_research_project, _ResearchSubject.primary_diagnosis_condition AS primary_diagnosis_condition, _ResearchSubject.primary_diagnosis_site AS primary_diagnosis_site, all_Subjects_v3_0_w_RS.id AS subject_id FROM gdc-bq-sample.dev.all_Subjects_v3_0_w_RS AS all_Subjects_v3_0_w_RS INNER JOIN UNNEST(all_Subjects_v3_0_w_RS.ResearchSubject) AS _ResearchSubject WHERE (UPPER(_ResearchSubject.primary_diagnosis_site) = UPPER('brain'))) as results WHERE rn = 1
            Offset: 0
            Count: 100
            Total Row Count: 2923
            More pages: True
            

Now we see that our 2314 subjects have 2923 researchsubjects between them, that means that some, but not all, of our subjects were participants in more than one study. Let's peek at the data:

In [8]:
researchsubjectresults.to_dataframe()

,id,identifier,member_of_research_project,primary_diagnosis_condition,primary_diagnosis_site,subject_id
0,01a13aba-74a4-4895-a5ad-e5119925c202,"[{'system': 'GDC', 'value': '01a13aba-74a4-489...",TCGA-LGG,Gliomas,Brain,TCGA-S9-A7IZ
1,104c1f4b-2139-11ea-aee1-0e1aae319e49,"[{'system': 'PDC', 'value': '104c1f4b-2139-11e...",CPTAC3-Discovery,Glioblastoma,Brain,C3L-01834
2,104c3685-2139-11ea-aee1-0e1aae319e49,"[{'system': 'PDC', 'value': '104c3685-2139-11e...",CPTAC3-Discovery,Glioblastoma,Brain,C3L-03392
3,1aa6b9ff-7c97-47d9-923f-2bd83f20e531,"[{'system': 'GDC', 'value': '1aa6b9ff-7c97-47d...",TCGA-LGG,Gliomas,Brain,TCGA-P5-A5EU
4,1f2f3bf6-acdd-4a72-8c33-230de40910eb,"[{'system': 'GDC', 'value': '1f2f3bf6-acdd-4a7...",TCGA-LGG,Gliomas,Brain,TCGA-RY-A843
...,...,...,...,...,...,...
95,d08d3049-ff5e-11e9-9a07-0a80fada099c,"[{'system': 'PDC', 'value': 'd08d3049-ff5e-11e...",Proteogenomic Analysis of Pediatric Brain Canc...,Pediatric/AYA Brain Tumors,Brain,C116850
96,d08d42ee-ff5e-11e9-9a07-0a80fada099c,"[{'system': 'PDC', 'value': 'd08d42ee-ff5e-11e...",Proteogenomic Analysis of Pediatric Brain Canc...,Pediatric/AYA Brain Tumors,Brain,C155103
97,d08d4986-ff5e-11e9-9a07-0a80fada099c,"[{'system': 'PDC', 'value': 'd08d4986-ff5e-11e...",Proteogenomic Analysis of Pediatric Brain Canc...,Pediatric/AYA Brain Tumors,Brain,C16974
98,d08d5e3c-ff5e-11e9-9a07-0a80fada099c,"[{'system': 'PDC', 'value': 'd08d5e3c-ff5e-11e...",Proteogenomic Analysis of Pediatric Brain Canc...,Pediatric/AYA Brain Tumors,Brain,C22140


Each row from the researchsubject endpoint results tells us about a subject in a given study. Using this endpoint we can find out information like what studies fit our search criteria, and also get data that we can filter to have only subjects from multiple studies, or only subjects from single studies.

Any given subject will have one row per study they participated in. The subject_id in the last column of this view is the same as the `id` in the first column of the Subjects endpoint results. You can use this to combine information across endpoints, which is covered in the [Merging Results](../MergingResults.ipynb) notebook.


---

<div style="background-color:#a2f2ed;color:black;padding:20px;">

<h3>ResearchSubject Field Definitions</h3>

<i>A researchsubject is a person/plant/animal/microbe within a given study. An individual who participates in 3 studies will have 3 researchsubject IDs</i>
    
<ul>
  <li><b>id:</b> The unique identifier for this researchsubject</li>
  <li><b>identifier:</b> An embedded array of information that includes the originating data center and the ID the researchsubject had there</li>
  <li><b>member_of_research_project:</b> The name of the study/project that the subject particpated in</li>
  <li><b>primary_diagnosis_condition:</b> The cancer, disease or other condition under study</li>
  <li><b>primary_diagnosis_site:</b> The anatomical location of the cancer or other condition in the subject</li>
  <li><b>subject_id:</b> An identifier for the subject. Can be joined to the `id` field from subject results</li>
</ul>  

</div>
    
---

### diagnosis

The diagnosis endpoint is an extension of the researchsubject endpoint, and returns information about research subjects that have a diagnosis that meets our search criteria:

In [9]:
diagnosisresults = myquery.diagnosis.run()
diagnosisresults.to_dataframe()

Getting results from database

Total execution time: 3455 ms


,id,identifier,primary_diagnosis,age_at_diagnosis,morphology,stage,grade,method_of_diagnosis,subject_id,researchsubject_id
0,030802cc-6775-551f-b388-680b89793c83,"[{'system': 'GDC', 'value': '030802cc-6775-551...","Oligodendroglioma, anaplastic",11528,9451/3,None,not reported,None,TCGA-FG-7638,1f54d21e-752f-496d-8453-4b9b8738cbce
1,12bcca94-ee52-5d03-900c-d775855a3775,"[{'system': 'GDC', 'value': '12bcca94-ee52-5d0...","Oligodendroglioma, NOS",18494,9450/3,None,not reported,None,TCGA-VM-A8C8,a66a3847-7570-4486-8845-86b6125104a2
2,22629218-d3b5-52b1-9a9c-f8db0feac530,"[{'system': 'GDC', 'value': '22629218-d3b5-52b...","Oligodendroglioma, NOS",24122,9450/3,None,not reported,None,TCGA-DU-6400,178ab87c-0582-4b02-bf6b-3a4c59a5ee1b
3,3bae4729-2848-11ec-b712-0a4e2186f121,"[{'system': 'PDC', 'value': '3bae4729-2848-11e...",Glioblastoma,15483,None,Not Reported,Not Reported,None,C3L-01142,104c14b2-2139-11ea-aee1-0e1aae319e49
4,3de321de-cd02-5183-b73f-f6ee62290d4c,"[{'system': 'GDC', 'value': '3de321de-cd02-518...",Glioblastoma,20813,9440/3,None,not reported,None,TCGA-02-0111,09ddf661-25fd-4baa-a1e8-bf90b2bb1d31
...,...,...,...,...,...,...,...,...,...,...
95,3e26d862-2848-11ec-b712-0a4e2186f121,"[{'system': 'PDC', 'value': '3e26d862-2848-11e...",Glioblastoma,21015,None,Not Reported,Not Reported,None,C3N-01369,104c53f6-2139-11ea-aee1-0e1aae319e49
96,6939f776-7ae7-5e1f-83f2-1a178cb17294,"[{'system': 'GDC', 'value': '6939f776-7ae7-5e1...",Glioblastoma,19591,9440/3,None,not reported,None,TCGA-06-6698,c129c34a-8d40-4d98-b9ad-a10d3f7b3ee0
97,78b0778d-5ed8-5c7f-a5a4-ec8b22e6b703,"[{'system': 'GDC', 'value': '78b0778d-5ed8-5c7...","Astrocytoma, anaplastic",8335,9401/3,None,not reported,None,TCGA-HT-A61B,cf2a1b98-5693-4c04-9167-f85e69a88a4a
98,b6d50cb3-6104-5cd7-a485-e1825e05aed9,"[{'system': 'GDC', 'value': 'b6d50cb3-6104-5cd...",Glioblastoma,16301,9440/3,None,not reported,None,TCGA-28-1747,35a7e49e-747b-45ce-9a07-3a5faba0ca94



---

<div style="background-color:#a2f2ed;color:black;padding:20px;">

<h3>Diagnosis Field Definitions</h3>

<i>A diagnosis is a medical classification of a disease for a given research subject in a given study. A single research subject may have different diagnoses across different studies</i>

    
<ul>
  <li><b>id:</b> The unique identifier for this diagnosis in this research subject</li>
  <li><b>identifier:</b> An embedded array of information that includes the originating data center and the ID the diagnosed researchsubject had there</li>
  <li><b>primary_diagnosis:</b> The main medical diagnosis for this subject in this study</li>
  <li><b>age_at_diagnosis:</b> The subjects age in days after birth on the day they were first diagnosed</li>
  <li><b>morphology:</b> The [International Classification of Diseases for Oncology](https://www.who.int/standards/classifications/other-classifications/international-classification-of-diseases-for-oncology) diagnosic code for this diagnosis</li>
  <li><b>stage:</b> A measure of disease spread. Different diseases may use different staging criteria, please refer to the originating data source to see what staging system is reported</li>
  <li><b>grade:</b> A measure of cell abnormality. Different diseases may use different grading criteria, please refer to the originating data source to see what grading system is reported</li>
  <li><b>method_of_diagnosis:</b> The test or system used for determining the diagnosis</li>
  <li><b>subject_id:</b> An identifier for the subject. Can be joined to the `id` field from subject results</li>
  <li><b>researchsubject_id</b> An identifier for the subject. Can be joined to the `id` field from researchsubject results</li>
</ul>  

</div>
    
---




### treatment

The treatment endpoint is an extension of diagnosis and returns information about treatments undertaken on research subjects that have a given diagnosis that meets our search criteria:

In [10]:
treatmentresults = myquery.treatment.run()
treatmentresults.to_dataframe()

Getting results from database

Total execution time: 3494 ms


,id,identifier,treatment_type,treatment_outcome,days_to_treatment_start,days_to_treatment_end,therapeutic_agent,treatment_anatomic_site,treatment_effect,treatment_end_reason,number_of_cycles,subject_id,researchsubject_id,researchsubject_diagnosis_id
0,197dce90-258c-57e0-92d0-2544e26ed9a6,"[{'system': 'GDC', 'value': '197dce90-258c-57e...","Radiation Therapy, NOS",None,None,None,None,None,None,None,None,TCGA-02-0079,e711a200-cdd5-459e-8dc0-1480bd9bc5df,3641075b-24dd-5c3b-a503-775a669b8c39
1,214b7976-ab54-55d7-8ef1-8e99e1fd2fa0,"[{'system': 'GDC', 'value': '214b7976-ab54-55d...","Pharmaceutical Therapy, NOS",None,None,None,None,None,None,None,None,TCGA-19-2629,e17a6048-7a72-42c0-ad3f-97cbff02bc9f,6cf67289-6980-5988-9ade-e70ea4cabeb5
2,28e42cf1-fd94-5f80-a67e-81e9aaab7de4,"[{'system': 'GDC', 'value': '28e42cf1-fd94-5f8...","Radiation Therapy, NOS",None,None,None,None,None,None,None,None,TCGA-02-0099,883dc176-925a-44f7-9ec8-e0cce33c2a54,155fc7a3-6178-5dba-91c3-b19c5ab7fe01
3,2db2f048-8a73-52bc-8436-12e88ecebdcb,"[{'system': 'GDC', 'value': '2db2f048-8a73-52b...","Pharmaceutical Therapy, NOS",None,None,None,None,None,None,None,None,TCGA-HW-A5KJ,8996dc64-8c5e-4e07-8ed8-2eb11c03626b,0237a1d9-6886-5f73-96db-a5d3146d6ba0
4,3526f3e3-c9a9-59e2-99af-c172ec127de9,"[{'system': 'GDC', 'value': '3526f3e3-c9a9-59e...","Radiation Therapy, NOS",None,None,None,None,None,None,None,None,TCGA-06-A5U1,81648978-a543-4eae-8848-98ddf1cb832b,751afd0f-9878-527b-94df-ce50d470962e
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,60f5ee59-7cda-5202-baea-58f5cc6d8968,"[{'system': 'GDC', 'value': '60f5ee59-7cda-520...","Radiation Therapy, NOS",None,None,None,None,None,None,None,None,TCGA-P5-A5EX,1f13065a-40f7-455d-b8eb-f9a128722eac,b6986923-4b45-50d3-a1ea-d1012cc0d4c0
96,6785995b-0255-589d-978f-4c8ef07fa751,"[{'system': 'GDC', 'value': '6785995b-0255-589...","Radiation Therapy, NOS",None,None,None,None,None,None,None,None,TCGA-06-0131,d75996d6-9f02-4478-a4a1-dfa7ab41de77,6244c026-492a-52e3-9837-775a6ce6fb86
97,7ad76eb0-cf0b-5f8a-98ae-fe034e6f0e12,"[{'system': 'GDC', 'value': '7ad76eb0-cf0b-5f8...","Pharmaceutical Therapy, NOS",None,None,None,None,None,None,None,None,TCGA-02-0057,7ee03602-bfec-413d-a05b-609cf60c6481,d44baa2a-bb72-5641-8bf0-afe8a031f5bf
98,7d9e8acd-8aa9-5c81-9a66-bfb9ec896b4e,"[{'system': 'GDC', 'value': '7d9e8acd-8aa9-5c8...","Pharmaceutical Therapy, NOS",None,None,None,None,None,None,None,None,TCGA-HT-8108,81fb338f-1023-49aa-ac0f-ba763a6cd68c,7619f026-b326-5dce-affd-50ecb15c78eb



---

<div style="background-color:#a2f2ed;color:black;padding:20px;">

<h3>Treatment Field Definitions</h3>

<i><i>A treatment is a medical intervention for a diagnosed disease in a given subject in a given study. A single research subject may have multiple treatments for a single diagnosis, and/or different diagnoses, and different treatments, across different studies</i></i>

    
<ul>
  <li><b>id:</b> The unique identifier for this treatment of this diagnosis in this research subject</li>
  <li><b>identifier:</b> An embedded array of information that includes the originating data center and the ID the treated researchsubject had there</li>
  <li><b>treatment_type:</b> The medical intervention undertaken</li>
  <li><b>treatment_outcome:</b> The result of the medical intervention</li>
  <li><b>days_to_treatment_start:</b> The [International Classification of Diseases for Oncology](https://www.who.int/standards/classifications/other-classifications/international-classification-of-diseases-for-oncology) diagnosic code for this diagnosis</li>
  <li><b>days_to_treatment_end:</b> A measure of disease spread. Different diseases may use different staging criteria, please refer to the originating data source to see what staging system is reported</li>
  <li><b>therapeutic_agent:</b> A measure of cell abnormality. Different diseases may use different grading criteria, please refer to the originating data source to see what grading system is reported</li>
  <li><b>treatment_anatomic_site:</b> The test or system used for determining the diagnosis</li>
  <li><b>treatment_effect:</b> The test or system used for determining the diagnosis</li>
  <li><b>treatment_end_reason:</b> The test or system used for determining the diagnosis</li>
  <li><b>number_of_cycles:</b> The test or system used for determining the diagnosis</li>
  <li><b>subject_id:</b> An identifier for the subject. Can be joined to the `id` field from subject results</li>
  <li><b>researchsubject_id</b> An identifier for the researchsubject. Can be joined to the `id` field from researchsubject results</li>
  <li><b>researchsubject_diagnosis_id</b> An identifier for the diagnosis. Can be joined to the `id` field from diagnosis results</li>
</ul>  

</div>
    
---


### file

The file endpoint returns information about files that meet our search criteria, regardless of whether they are attached to subjects, research-subjects or specimens:

### specimens

We can use this same query to see what specimens are available for brain tissue at the CDA:

In [11]:
specresults =  myquery.specimen.run()
print(specresults)

Getting results from database

Total execution time: 3535 ms

            QueryID: f2484240-9267-4ce3-b3ba-d335fd1e6284
            Query:SELECT results.* EXCEPT(rn) FROM (SELECT ROW_NUMBER() OVER (PARTITION BY _ResearchSubject_Specimen.id, all_Subjects_v3_0_w_RS.id, _ResearchSubject.id) as rn, _ResearchSubject_Specimen.id AS id, _ResearchSubject_Specimen.identifier AS identifier, _ResearchSubject_Specimen.associated_project AS associated_project, _ResearchSubject_Specimen.age_at_collection AS age_at_collection, _ResearchSubject_Specimen.primary_disease_type AS primary_disease_type, _ResearchSubject_Specimen.anatomical_site AS anatomical_site, _ResearchSubject_Specimen.source_material_type AS source_material_type, _ResearchSubject_Specimen.specimen_type AS specimen_type, _ResearchSubject_Specimen.derived_from_specimen AS derived_from_specimen, all_Subjects_v3_0_w_RS.id AS subject_id, _ResearchSubject.id AS researchsubject_id FROM gdc-bq-sample.dev.all_Subjects_v3_0_w_RS AS all_Subjects_v3_0_w_RS INNER JOIN UNNEST(al

Nearly 40,000 specimens meet our search criteria! We would typically expect this number to be much larger than our number of subjects or researchsubjects. First because studies will often take more than one sample per subject, and second because any given specimen might be aliquoted out to be used in multiple tests. Since we didn't specify any further filters, our results will return all of these as seperate speciments. Let's look at a few:

In [12]:
specresults.to_dataframe()

,id,identifier,associated_project,age_at_collection,primary_disease_type,anatomical_site,source_material_type,specimen_type,derived_from_specimen,subject_id,researchsubject_id
0,00d54cc2-96ec-4fab-9239-b6253b7dd637,"[{'system': 'GDC', 'value': '00d54cc2-96ec-4fa...",TCGA-GBM,-13116,Gliomas,None,Primary Tumor,aliquot,2b9a6333-4802-4d89-9164-1a30754b243b,TCGA-02-0024,0553e60e-3510-417d-af8a-75947ebe8ab6
1,01be801d-c376-458a-ad95-15dcbe474113,"[{'system': 'GDC', 'value': '01be801d-c376-458...",TCGA-GBM,-24502,Gliomas,None,Primary Tumor,slide,695d8016-14a7-5e09-b7e9-7466ce7685ae,TCGA-06-0127,f66d92ff-85ad-4c83-b127-ce34c8488040
2,02876b00-a460-4a00-af72-f227e199b73f,"[{'system': 'GDC', 'value': '02876b00-a460-4a0...",TCGA-LGG,-13371,Gliomas,None,Primary Tumor,slide,5a178f84-37c6-419a-b9f3-9e71550a3f8e,TCGA-DB-5275,bbfb5399-8d43-4b75-bf90-23ec142697d7
3,028b33a2-4b77-4b68-baa3-e756946dd085,"[{'system': 'GDC', 'value': '028b33a2-4b77-4b6...",TCGA-LGG,-11509,Gliomas,None,Primary Tumor,aliquot,e8b02631-1c0e-4509-a74f-a278078b187a,TCGA-CS-4938,334f715e-08dc-4a29-b8e4-b010b829c478
4,02f3955e-590c-53ac-af30-65e711fcfed4,"[{'system': 'GDC', 'value': '02f3955e-590c-53a...",TCGA-LGG,-10342,Gliomas,None,Primary Tumor,portion,5545fec5-5655-4c59-aee5-f5fd4ef6e866,TCGA-S9-A7R3,b7395082-415b-4ab3-bf1e-59d0f4e4b488
...,...,...,...,...,...,...,...,...,...,...,...
95,3a71b861-5ede-4489-a161-8583f66e7e05,"[{'system': 'GDC', 'value': '3a71b861-5ede-448...",TCGA-GBM,-20449,Gliomas,None,Blood Derived Normal,aliquot,f8a76522-1fae-4597-a907-4becabbf5744,TCGA-06-0645,878584ad-e6b6-493a-9f7f-3e284f5d9f68
96,3abd8a28-af49-5c9c-9ab3-441cb5fb1432,"[{'system': 'GDC', 'value': '3abd8a28-af49-5c9...",CPTAC-3,-12827,Gliomas,None,Primary Tumor,analyte,8a69e556-9c23-5b00-9a7e-ce8cb9574be2,C3N-03184,f9a63d56-94cb-4f15-ac35-7c59a08f4104
97,3b695a86-1603-4a6a-81d7-4b91f219b2a4,"[{'system': 'GDC', 'value': '3b695a86-1603-4a6...",TCGA-LGG,-10170,Gliomas,None,Primary Tumor,aliquot,cc17a5fd-ad5a-4b7c-80de-2d35dfc65348,TCGA-DU-A6S7,a56c2083-33d5-4c98-b85f-3b68a3d80add
98,3c1d256c-58d0-525e-b157-aa3063a09bb1,"[{'system': 'GDC', 'value': '3c1d256c-58d0-525...",HCMI-CMDC,-21192,Gliomas,None,Expanded Next Generation Cancer Model,portion,fdbc5c4f-c278-4f88-9015-cf06af5db153,HCM-BROD-0029-C71,b5160217-33c3-47cd-8540-44ca283c8464



---

<div style="background-color:#a2f2ed;color:black;padding:20px;">

<h3>Specimen Field Definitions</h3>

<i>A specimen is a tissue sample taken from a given subject, or a portion of the original sample. A given specimen will have only a single subject ID and a single research subject ID</i>

    
<ul>
  <li><b>id:</b> The unique identifier for this specimen</li>
  <li><b>identifier:</b> An embedded array of information that includes the originating data center and the ID the specimen had there</li>
  <li><b>associated_project:</b> The name of the study/project that the subject particpated in</li>
  <li><b>age_at_collection:</b> The subjects age in days (counting backwards to birth) on the day of the collection of the proximate specimen</li>
  <li><b>primary_disease_type:</b> The cancer, disease or other condition under study</li>
  <li><b>anatomical_site:</b> The body part from which the proximate specimen was taken</li>
  <li><b>source_material_type:</b> What type of tissue the specimen consists of</li>
  <li><b>specimen_type:</b> One of: analyte, aliquot, portion, sample, or slide</li>
  <li><b>derived_from_specimen:</b> For derived samples, the `id` for the original sample</li>
  <li><b>subject_id:</b> An identifier for the subject. Can be joined to the `id` field from subject results</li>
  <li><b>researchsubject_id</b> An identifier for the subject. Can be joined to the `id` field from researchsubject results</li>
</ul>  

</div>
    
---


### file

The file endpoint returns information about files that meet our search criteria, regardless of whether they are attached to subjects, research-subjects or specimens: 

In [13]:
myquery.file.run()

Getting results from database

Total execution time: 3536 ms



            QueryID: 25256170-6f42-4a02-b2ed-ceb837aa205f
            Query:with ResearchSubject_Specimen_files as (SELECT results.* EXCEPT(rn) FROM (SELECT ROW_NUMBER() OVER (PARTITION BY all_Files_v3_0_w_RS.id, _ResearchSubject.id, _ResearchSubject_Specimen.id, all_Subjects_v3_0_w_RS.id) as rn, all_Files_v3_0_w_RS.id AS id, all_Files_v3_0_w_RS.identifier AS identifier, all_Files_v3_0_w_RS.label AS label, all_Files_v3_0_w_RS.data_category AS data_category, all_Files_v3_0_w_RS.data_type AS data_type, all_Files_v3_0_w_RS.file_format AS file_format, all_Files_v3_0_w_RS.associated_project AS associated_project, all_Files_v3_0_w_RS.drs_uri AS drs_uri, all_Files_v3_0_w_RS.byte_size AS byte_size, all_Files_v3_0_w_RS.checksum AS checksum, all_Files_v3_0_w_RS.data_modality AS data_modality, all_Files_v3_0_w_RS.imaging_modality AS imaging_modality, all_Files_v3_0_w_RS.dbgap_accession_number AS dbgap_accession_number, _ResearchSubject_Specimen.id AS researchsubject_specimen_id, _ResearchSubject

In [14]:
fileresults = myquery.file.run()
fileresults.to_dataframe()

Getting results from database

Total execution time: 3564 ms


,id,identifier,label,data_category,data_type,file_format,associated_project,drs_uri,byte_size,checksum,data_modality,imaging_modality,dbgap_accession_number,researchsubject_specimen_id,researchsubject_id,subject_id
0,4ffcd65e-53a9-460f-8bd4-9b2360cbe85f,"[{'system': 'IDC', 'value': '4ffcd65e-53a9-460...",idc/4ffcd65e-53a9-460f-8bd4-9b2360cbe85f.dcm,Imaging,None,DICOM,ivygap,drs://dg.4DFC:4ffcd65e-53a9-460f-8bd4-9b2360cb...,None,None,Imaging,MR,None,,W5__ivygap,W5
1,55622e8e-866e-4c70-a7bf-b70150142acc,"[{'system': 'IDC', 'value': '55622e8e-866e-4c7...",idc/55622e8e-866e-4c70-a7bf-b70150142acc.dcm,Imaging,None,DICOM,qin_gbm_treatment_response,drs://dg.4DFC:55622e8e-866e-4c70-a7bf-b7015014...,None,None,Imaging,MR,None,,QIN-GBM-TR-04__qin_gbm_treatment_response,QIN-GBM-TR-04
2,5a878629-133b-4fe6-8a2c-0968933a3769,"[{'system': 'IDC', 'value': '5a878629-133b-4fe...",idc/5a878629-133b-4fe6-8a2c-0968933a3769.dcm,Imaging,None,DICOM,tcga_lgg,drs://dg.4DFC:5a878629-133b-4fe6-8a2c-0968933a...,None,None,Imaging,MR,None,,TCGA-DU-8166__tcga_lgg,TCGA-DU-8166
3,754f5cd3-6ffe-4de7-9302-4a2bee6d4f2c,"[{'system': 'IDC', 'value': '754f5cd3-6ffe-4de...",idc/754f5cd3-6ffe-4de7-9302-4a2bee6d4f2c.dcm,Imaging,None,DICOM,acrin_fmiso_brain,drs://dg.4DFC:754f5cd3-6ffe-4de7-9302-4a2bee6d...,None,None,Imaging,MR,None,,ACRIN-FMISO-Brain-031__acrin_fmiso_brain,ACRIN-FMISO-Brain-031
4,7c8bab39-7554-4ac4-8161-f431cd449815,"[{'system': 'IDC', 'value': '7c8bab39-7554-4ac...",idc/7c8bab39-7554-4ac4-8161-f431cd449815.dcm,Imaging,None,DICOM,qin_gbm_treatment_response,drs://dg.4DFC:7c8bab39-7554-4ac4-8161-f431cd44...,None,None,Imaging,MR,None,,QIN-GBM-TR-41__qin_gbm_treatment_response,QIN-GBM-TR-41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0dabdcbc-628c-4cbe-b3d6-08537f9571b4,"[{'system': 'IDC', 'value': '0dabdcbc-628c-4cb...",idc/0dabdcbc-628c-4cbe-b3d6-08537f9571b4.dcm,Imaging,None,DICOM,qin_gbm_treatment_response,drs://dg.4DFC:0dabdcbc-628c-4cbe-b3d6-08537f95...,None,None,Imaging,MR,None,,QIN-GBM-TR-49__qin_gbm_treatment_response,QIN-GBM-TR-49
96,2f974394-0f23-49ce-8d5a-dd02819609fb,"[{'system': 'IDC', 'value': '2f974394-0f23-49c...",idc/2f974394-0f23-49ce-8d5a-dd02819609fb.dcm,Imaging,None,DICOM,acrin_dsc_mr_brain,drs://dg.4DFC:2f974394-0f23-49ce-8d5a-dd028196...,None,None,Imaging,MR,None,,ACRIN-DSC-MR-Brain-042__acrin_dsc_mr_brain,ACRIN-DSC-MR-Brain-042
97,436b1790-3c96-451f-8adc-281dfb1bd10c,"[{'system': 'IDC', 'value': '436b1790-3c96-451...",idc/436b1790-3c96-451f-8adc-281dfb1bd10c.dcm,Imaging,None,DICOM,qin_gbm_treatment_response,drs://dg.4DFC:436b1790-3c96-451f-8adc-281dfb1b...,None,None,Imaging,MR,None,,QIN-GBM-TR-42__qin_gbm_treatment_response,QIN-GBM-TR-42
98,063157d2-7b14-4a94-b526-a64338ad27cb,"[{'system': 'IDC', 'value': '063157d2-7b14-4a9...",idc/063157d2-7b14-4a94-b526-a64338ad27cb.dcm,Imaging,None,DICOM,ivygap,drs://dg.4DFC:063157d2-7b14-4a94-b526-a64338ad...,None,None,Imaging,MR,None,,W4__ivygap,W4


As you might expect, searching file gives us a huge number of results. This is great if you are surveying what kind of data is available, but is less useful for getting a coherent cohort. 

A better way to get files for a specific cohort is to chain your queries together, which we cover in the next tutorial [Chaining Queries](../ChainingQueries): Combine information from multiple endpoints, and build And/Or/Like and other advanced query strings.

Another useful way to look at high level information is to use our counts feature which returns summary information rather than the full search results. Check out the [Data Summaries tutorial](../DataSummaries) to try it.



---

<div style="background-color:#a2f2ed;color:black;padding:20px;">

<h3>File Field Definitions</h3>

<i>A unit of data about subjects, researchsubjects, specimens, or their associated information</i>

    
<ul>
  <li><b>id:</b> The unique identifier for this file</li>
  <li><b>identifier:</b> An embedded array of information that includes the originating data center and the ID the file had there</li>
  <li><b>label:</b> The full name of the file</li>
  <li><b>data_catagory:</b> A desecription of the kind of general kind data the file holds</li>
  <li><b>data_type:</b> A more specific descripton of the data type</li>
  <li><b>file_format:</b> The extension of the file</li>
  <li><b>associated_project:</b> The name the data center uses for the study this file was generated for</li>
  <li><b>drs_uri:</b> A unique identifier that can be used to retreive this specific file from a server</li>
  <li><b>byte_size:</b> Size of the file in bytes</li>
  <li><b>checksum:</b> The md5 value for the file</li>
  <li><b>data_modality:</b> A high level descriptor of file data, always one of "Genomic", "Proteomic", or "Imaging"</li>
  <li><b>imaging_modality</b> For files with the `data_modality` of "Imaging", a descriptor for the image type</li>
  <li><b>dbgap_accession_number:</b> The project id number for this data on dbGaP</li>
</ul>  

</div>
    
---
